<a href="https://colab.research.google.com/github/mafaldasalomao/Transfering_learning_TF-ExtractFeatures/blob/main/04_transfering_learning_TF_partI_Feature_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
#download  the data
# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzip the downloaded data
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2022-10-07 19:11:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   223MB/s    in 0.7s    

2022-10-07 19:11:41 (223 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [3]:
#how many images in each flder?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories in {len(filenames)} in the {dirpath}.")

There are 2 directories in 0 in the 10_food_classes_10_percent.
There are 10 directories in 0 in the 10_food_classes_10_percent/train.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/ice_cream.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/steak.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/hamburger.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/pizza.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/fried_rice.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/sushi.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/chicken_curry.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/grilled_salmon.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/chicken_wings.
There are 0 directories in 75 in the 10_food_classes_10_percent/train/ramen.
There are 10 directories in 0 in the 10_food_classes_10_p

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
